<a href="https://colab.research.google.com/github/varat21/coursework/blob/main/coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

file_path = '/content/drive/My Drive/dataset.csv'

# Load the dataset
data = pd.read_csv(file_path)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as dt
from sklearn.preprocessing import LabelEncoder


# Convert to DataFrame (if not already)
data = pd.DataFrame(data)

# Print the data
print("data")
print(data)

label_encoder = LabelEncoder()

# Features and target column
datainX = data.iloc[:, 1:20]  # Features (X)
datainY = data.iloc[:, 20]    # Target (Y)

# Print column names and target data
print(datainX.columns)
print("debug y")
print(datainY)

# Encode the target column (datainY)
datainY = label_encoder.fit_transform(datainY)

# Print the encoded target column
print("Encoded datainY:")
print(datainY)

# Specify the columns to encode in datainX
columns_to_encode = [0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
columns_to_remove=[0,5,6,7,12,13]

# Encode specified columns in datainX
for col_index in columns_to_encode:
    column_name = datainX.columns[col_index]
    datainX[column_name] = label_encoder.fit_transform(datainX[column_name])

# Print the encoded features and target
print("Encoded datainX:")
print(datainX.shape)
print("Encoded datainY:")
print(datainY.shape)

datainX = datainX.drop(columns=datainX.columns[columns_to_remove])



# Convert datainY to DataFrame for concatenation
datainY = pd.DataFrame(datainY)

# Concatenate datainX and datainY along columns (axis=1)

# Check the data types of columns in datainX
datainX['TotalCharges'].replace(' ', '0.0', inplace=True)

# Convert the column to float
datainX['TotalCharges'] = datainX['TotalCharges'].astype(float)

# Check the result
print(datainX['TotalCharges'])
for_corr = pd.concat([datainX, datainY], axis=1)
corr_matrix=for_corr.corr()
print(corr_matrix)


print(datainX.columns)
cols=datainX.columns

import itertools as iteration
combinations_prepared=list(iteration.combinations(cols,4))
# print(combinations_prepared)

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as dt

dt=dt(max_depth=21)

from sklearn.preprocessing import StandardScaler

# Z-score normalization
scaler = StandardScaler()
datainX = pd.DataFrame(scaler.fit_transform(datainX), columns=datainX.columns)


X_train,X_test,Y_train,Y_test=train_test_split(datainX,datainY, test_size=0.25,shuffle=True)
dt.fit(X_train,Y_train)

test_predictions=[]

for combo in combinations_prepared:
    X_tr=X_train[list(combo)]
    X_te=X_test[list(combo)]
    dt.fit(X_tr,Y_train)

    predict=dt.predict(X_te)
    test_predictions.append(predict)

test_predictions=pd.DataFrame(test_predictions).T

test_predictions=test_predictions.mean(axis=1)
test_predictions = (test_predictions > 0.5).astype(int)

from sklearn.metrics import mean_squared_error, mean_absolute_error, classification_report, accuracy_score

# Assuming `test_predictions` contains binary predictions (0 or 1)
# and `Y_train` contains the true labels for comparison

print(test_predictions)
# Mean Squared Error (MSE)
mse = mean_squared_error(Y_test, test_predictions)
print(f"Mean Squared Error (MSE): {mse}")

# Mean Absolute Error (MAE)
mae = mean_absolute_error(Y_test, test_predictions)
print(f"Mean Absolute Error (MAE): {mae}")

# Classification Report
print("Classification Report:")
print(classification_report(Y_test, test_predictions))

# Accuracy Score
accuracy = accuracy_score(Y_test, test_predictions)
print(f"Accuracy Score: {accuracy}")


import numpy as np
from sklearn.tree import DecisionTreeRegressor
from time import time

class GradientBoostingStepByStep:
    def __init__(self, n_estimators=10000, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.initial_prediction = None
        self.training_errors = []

    def fit(self, X, y):

        y = np.ravel(y)


        self.initial_prediction = np.mean(y)
        print(f"Initial prediction (mean of y): {self.initial_prediction:.4f}")


        current_predictions = np.full(len(y), self.initial_prediction, dtype=float)

        for iteration in range(self.n_estimators):

            residuals = y - current_predictions


            mse = np.mean(residuals ** 2)
            self.training_errors.append(mse)


            if (iteration + 1) % 10 == 0:
                elapsed_time = time() - start_time
                print(f"Iteration {iteration + 1}/100 | MSE: {mse:.4f} | Time elapsed: {elapsed_time:.2f}s")


            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            tree_predictions = tree.predict(X)

            current_predictions += self.learning_rate * tree_predictions
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.full(X.shape[0], self.initial_prediction)
        for tree in self.trees:
            predictions += self.learning_rate * tree.predict(X)
        return predictions


Y_train = np.ravel(Y_train)
Y_test = np.ravel(Y_test)

print("=" * 60)


gb_model = GradientBoostingStepByStep(n_estimators=10000, learning_rate=0.1, max_depth=3)
start_time = time()
gb_model.fit(X_train, Y_train)
total_time = time() - start_time


train_preds = gb_model.predict(X_train)
test_preds = gb_model.predict(X_test)

# Calculate final metrics
train_mse = np.mean((Y_train - train_preds) ** 2)
test_mse = np.mean((Y_test - test_preds) ** 2)

print("\nTraining Complete!")
print("=" * 60)
print(f"Total training time: {total_time:.2f} seconds")
print(f"Final Training MSE: {train_mse:.4f}")
print(f"Final Testing MSE: {test_mse:.4f}")
print(f"MSE Improvement: {gb_model.training_errors[0] - train_mse:.4f}")

# Print error reduction over time
print("\nError reduction at different stages:")
print(f"Start MSE: {gb_model.training_errors[0]:.4f}")
print(f"25% complete MSE: {gb_model.training_errors[24]:.4f}")
print(f"50% complete MSE: {gb_model.training_errors[49]:.4f}")
print(f"75% complete MSE: {gb_model.training_errors[74]:.4f}")
print(f"Final MSE: {gb_model.training_errors[-1]:.4f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
data
      customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
8907  1699-HPSBG    Male              0      No         No      12   
8908  8775-CEBBJ  Female              0      No         No       9   
8909  6894-LFHLY    Male              1      No         No       1   
8910  0639-TSIQW  Female              0      No         No      67   
8911  8361-LTMKD    Male              1     Yes         No       4   

     PhoneService     Mul

<ipython-input-11-09ceafc579ee>:68: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datainX['TotalCharges'].replace(' ', '0.0', inplace=True)


0       1
1       0
2       0
3       1
4       1
       ..
2223    0
2224    0
2225    1
2226    1
2227    0
Length: 2228, dtype: int64
Mean Squared Error (MSE): 0.16292639138240575
Mean Absolute Error (MAE): 0.16292639138240575
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.81      0.85      1271
           1       0.78      0.87      0.82       957

    accuracy                           0.84      2228
   macro avg       0.84      0.84      0.84      2228
weighted avg       0.84      0.84      0.84      2228

Accuracy Score: 0.8370736086175943
Starting gradient boosting training with 100 iterations...
Initial prediction (mean of y): 0.4161
Iteration 10/100 | MSE: 0.1751 | Time elapsed: 0.11s
Iteration 20/100 | MSE: 0.1580 | Time elapsed: 0.23s
Iteration 30/100 | MSE: 0.1518 | Time elapsed: 0.36s
Iteration 40/100 | MSE: 0.1489 | Time elapsed: 0.48s
Iteration 50/100 | MSE: 0.1472 | Time elapsed: 0.59s
Iteration 60/100 | MSE

In [16]:
test_preds=(test_preds>0.5).astype(int)
print((test_preds>0.5).astype(int))
print(Y_test)
test_mse = np.mean((Y_test - test_preds) ** 2)
print(test_mse)

[1 1 0 ... 1 1 0]
[1 1 0 ... 0 1 0]
0.15933572710951527


In [17]:
train_preds_binary = (train_preds > 0.5).astype(int)
print("\nTraining Set:")
print("Binary Predictions:", train_preds_binary)
print("Actual Values:", Y_train)
train_mse = np.mean((Y_train - train_preds_binary) ** 2)
print("Train MSE:", train_mse)


Training Set:
Binary Predictions: [1 1 0 ... 0 0 0]
Actual Values: [1 1 0 ... 0 0 0]
Train MSE: 0.0034410532615200477


In [33]:
from sklearn.metrics import mean_squared_error

# Initialize variables
mean_value = np.mean(Y_train)
initial_preds = np.full(Y_train.shape, mean_value)

finaltrees_append = []  # List to store decision tree models
learning_rate = 0.1    # Learning rate as a variable

# Create initial predictions
preds_inloop = initial_preds.copy()

# Track best validation score
best_val_score = float('inf')
patience = 5  # Number of iterations to wait before early stopping
patience_counter = 0

for iteration in range(1, 10000):
    # Compute residuals
    residuals = Y_train - preds_inloop

    # Create a new tree instance each iteration
    dt = DecisionTreeRegressor(max_depth=2)

    # Fit the decision tree on residuals
    dt.fit(X_train, residuals)

    # Predict using the current tree
    train_pred = dt.predict(X_train)

    # Update predictions
    preds_inloop += train_pred * learning_rate

    # Store the tree
    finaltrees_append.append(dt)

    # Calculate train MSE
    train_mse = mean_squared_error(Y_train, preds_inloop)

    # Print progress
    print(f"Iteration {iteration}:")
    print(f"  Train residual mean: {np.mean(residuals):.6f}")
    print(f"  Train MSE: {train_mse:.6f}")

    # Early stopping (optional)
    if train_mse < best_val_score:
        best_val_score = train_mse
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break

# Final Predictions
final_predictions = preds_inloop
final_predictions=(final_predictions>0.5).astype(int)
# Compare predicted values with Y_train
print("Final Train MSE:", mean_squared_error(Y_train, final_predictions))
print("Final Train MAE:", np.mean(np.abs(Y_train - final_predictions)))


Streaming output truncated to the last 5000 lines.
Iteration 8334:
  Train residual mean: -0.000000
  Train MSE: 0.052893
Iteration 8335:
  Train residual mean: -0.000000
  Train MSE: 0.052888
Iteration 8336:
  Train residual mean: -0.000000
  Train MSE: 0.052887
Iteration 8337:
  Train residual mean: -0.000000
  Train MSE: 0.052886
Iteration 8338:
  Train residual mean: -0.000000
  Train MSE: 0.052886
Iteration 8339:
  Train residual mean: -0.000000
  Train MSE: 0.052865
Iteration 8340:
  Train residual mean: -0.000000
  Train MSE: 0.052859
Iteration 8341:
  Train residual mean: -0.000000
  Train MSE: 0.052858
Iteration 8342:
  Train residual mean: -0.000000
  Train MSE: 0.052854
Iteration 8343:
  Train residual mean: -0.000000
  Train MSE: 0.052847
Iteration 8344:
  Train residual mean: -0.000000
  Train MSE: 0.052847
Iteration 8345:
  Train residual mean: -0.000000
  Train MSE: 0.052845
Iteration 8346:
  Train residual mean: -0.000000
  Train MSE: 0.052843
Iteration 8347:
  Train re

In [34]:
final_predictions=(final_predictions>0.5).astype(int)
# Compare predicted values with Y_train
print("Final Train MSE:", mean_squared_error(Y_train, final_predictions))
print("Final Train MAE:", np.mean(np.abs(Y_train - final_predictions)))

Final Train MSE: 0.03426092160383004
Final Train MAE: 0.03426092160383004
